# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys

# Call models
from src.call_models import google_connect, call_gemini, all_string_gemini_config, all_int_gemini_config
from src.translate_func import gemini_translation, gemini_classification

# Datasets

from src.benchmarks_code import hellaswag
from prompts import hellaswag_prompts

# Access keys
from my_access_keys import google_access_key

# .csv utils
from src.save_utils import add_dataset_to_csv

# Remove annoying warning
from IPython.core.display_functions import display

# GetDataset

In [6]:
google_client = google_connect(google_access_key)

In [16]:
hellaswag_dataset = hellaswag.get_hellaswag_datasets()
hellaswag_dataset

hellaswag_file_name = 'compare_csv/hellaswag/hellaswag_test_20_samples.csv'

print(hellaswag_file_name)
hellaswag_dataset

compare_csv/hellaswag/hellaswag_test_20_samples.csv


{'hellaswag_val': Dataset({
     features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
     num_rows: 10042
 })}

In [17]:
hesw_text = hellaswag_dataset['hellaswag_val'].select(range(0, 1_000, 30))
hesw_text = hesw_text.take(20)
hesw_text

Dataset({
    features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
    num_rows: 20
})

In [19]:
df = pd.read_csv(hellaswag_file_name)
text_df = pd.read_csv(hellaswag_file_name[:-4] + '-text.csv')
print(df.shape, text_df.shape)
display(df.head(2))
display(text_df.head(2))

(20, 1) (20, 1)


,original
0,<activity_label>Roof shingle removal</activity...
1,<activity_label>Gargling mouthwash</activity_l...


,original
0,<activity_label>Roof shingle removal</activity...
1,<activity_label>Gargling mouthwash</activity_l...


# Classify whether to translate or not

In [110]:
exp_name = 'gemini_think_classify'

labels, text_output = gemini_classification(
    google_client,
    {'run_on': hesw_text},
    hellaswag_prompts.HELLASWAG_CLASSIFICATION_GEMINI,
    hellaswag_prompts.HELLASWAG_CLASSIFICATION_FEW_SHOTS,
    hellaswag.hellaswag_sample_to_dict,
    hellaswag.hellaswag_dict_to_sample,
    think_bud=128,
)

labels, text_output = labels['run_on'], text_output['run_on']

Classifying run_on...


  0%|          | 0/34 [00:00<?, ?it/s]

In [111]:
pd.Series(labels).value_counts()

Universal           22
Can be localized     8
Foreign              4
Name: count, dtype: int64

In [106]:
pd.Series(labels).value_counts()

Universal           19
Can be localized    11
Foreign              4
Name: count, dtype: int64

In [107]:
from src.translate_func import dict_to_prompt

In [108]:
for cnt, (s, l) in enumerate(zip(hesw_text, labels)):
    print(f"{cnt:4} | {s['activity_label']:30} | {l:25}")

   0 | Wakeboarding                   | Can be localized         
   1 | Horseback riding               | Universal                
   2 | Using the balance beam         | Universal                
   3 | Playing racquetball            | Can be localized         
   4 | Changing car wheel             | Universal                
   5 | Snowboarding                   | Can be localized         
   6 | Doing nails                    | Universal                
   7 | Riding bumper cars             | Universal                
   8 | Playing ten pins               | Universal                
   9 | Rafting                        | Can be localized         
  10 | Longboarding                   | Can be localized         
  11 | Using uneven bars              | Can be localized         
  12 | Braiding hair                  | Universal                
  13 | Snowboarding                   | Foreign                  
  14 | Pole vault                     | Can be localized         
  15 | Usi

In [109]:
for cnt, (s, l) in enumerate(zip(hesw_text, labels)):
    print(f'Example {cnt}:')
    print(f'Classification: {l}\n')
    # print(s['activity_label'])
    print(dict_to_prompt(hellaswag.hellaswag_sample_to_dict(s)))
    print('\n' + '-' * 50 + '\n')

Example 0:
Classification: Can be localized

<activity_label>Wakeboarding</activity_label>
<ctx_a>A man is being pulled on a water ski as he floats in the water casually.</ctx_a>
<ctx_b>he</ctx_b>
<ctx>A man is being pulled on a water ski as he floats in the water casually. he</ctx>
<ending 1>mounts the water ski and tears through the water at fast speeds.</ending 1>
<ending 2>goes over several speeds, trying to stay upright.</ending 2>
<ending 3>struggles a little bit as he talks about it.</ending 3>
<ending 4>is seated in a boat with three other people.</ending 4>

--------------------------------------------------

Example 1:
Classification: Universal

<activity_label>Horseback riding</activity_label>
<ctx_a>These horses are walking in a grassy area by themselves and then a woman is shown brushing the horse's hair.</ctx_a>
<ctx_b>the camera</ctx_b>
<ctx>These horses are walking in a grassy area by themselves and then a woman is shown brushing the horse's hair. the camera</ctx>
<endi

In [77]:
print(text_output[9])

**Rafting in Israel: A Universal Experience**

Okay, so I'm presented with this "HellaSwag" example: rafting. The goal is to figure out its cultural relevance within Israel. Honestly, this one's pretty straightforward.  Let's break it down:

First, what *is* rafting? Just as I'd expect my user to know, it's a recreational water activity using inflatable rafts. And yes, in Israel, rafting is *hugely* popular, especially on the Jordan River. It's almost synonymous with summer fun for families, youth groups, and tourists.

The context describes dipping under a waterfall – which, of course, Israel has its own share of!  The social aspect, posing with fellow rafters, is entirely relatable. The description is remarkably generic and perfectly captures the essence of rafting as it's experienced here. No foreign elements jump out.

Therefore, my conclusion? This is a "Universal" concept. It's instantly recognizable and completely understandable to an Israeli audience.  Rafting is a common, well

# Translate

In [20]:
hesw_text

Dataset({
    features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
    num_rows: 20
})

In [21]:
exp_name = 'gemini'

hebrew_datasets, text_output = gemini_translation(
    google_client,
    {'run': hesw_text},
    hellaswag_prompts.HELLASWAG_INSTRUCT_V1_GEMINI,
    hellaswag_prompts.HELLASWAG_FEW_SHOTS,
    hellaswag.hellaswag_sample_to_dict,
    hellaswag.hellaswag_dict_to_sample,
    if_pro=True,
    think_bud=128,
)

hebrew_datasets = hebrew_datasets['run']
text_output = text_output['run']
hebrew_datasets

Translating run...


  0%|          | 0/20 [00:00<?, ?it/s]

Dataset({
    features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'translation_status'],
    num_rows: 20
})

In [22]:
hebrew_datasets[0]

{'ind': 24,
 'activity_label': 'הסרת רעפים מהגג',
 'ctx_a': 'גבר יושב על גג.',
 'ctx_b': 'הוא',
 'ctx': 'גבר יושב על גג. הוא',
 'endings': ['משתמש בניילון נצמד כדי לעטוף זוג מגלשי סקי.',
  'תולש אריחים ישרים.',
  'מחזיק קובייה הונגרית.',
  'מתחיל לתלוש את הרעפים מהגג.'],
 'source_id': 'activitynet~v_-JhWjGDPHMY',
 'split': 'val',
 'split_type': 'indomain',
 'label': '3',
 'translation_status': 'Success'}

In [23]:
df = add_dataset_to_csv(hellaswag_file_name, exp_name, hebrew_datasets, hellaswag.hellaswag_sample_to_dict)
text_df[exp_name + ' text'] = text_output
text_df.to_csv(hellaswag_file_name[:-4] + '-text.csv', index=False)
print(df.shape, text_df.shape)
display(df.head(2))
display(text_df.head(2))

(20, 2) (20, 2)


,original,gemini
0,<activity_label>Roof shingle removal</activity...,<activity_label>הסרת רעפים מהגג</activity_labe...
1,<activity_label>Gargling mouthwash</activity_l...,<activity_label>גרגור מי פה</activity_label>\n...


,original,gemini text
0,<activity_label>Roof shingle removal</activity...,**My Translation Task: English to Hebrew JSON*...
1,<activity_label>Gargling mouthwash</activity_l...,"**Translating ""Gargling Mouthwash"" into Hebrew..."


In [26]:
df['answer_label'] = pd.Series(hebrew_datasets['label'])

In [27]:
df.head()

,original,gemini,answer_label
0,<activity_label>Roof shingle removal</activity...,<activity_label>הסרת רעפים מהגג</activity_labe...,3
1,<activity_label>Gargling mouthwash</activity_l...,<activity_label>גרגור מי פה</activity_label>\n...,1
2,<activity_label>Kayaking</activity_label>\n<ct...,<activity_label>שייט בקיאק</activity_label>\n<...,1
3,<activity_label>Surfing</activity_label>\n<ctx...,<activity_label>גלישת גלים</activity_label>\n<...,1
4,<activity_label>Playing violin</activity_label...,<activity_label>נגינה בכינור</activity_label>\...,2


In [34]:
df.to_csv(hellaswag_file_name, index=False)